<a href="https://colab.research.google.com/github/WafulaNasombi/phishin_detection/blob/main/Phishing_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to Google Drive

In [25]:
#connect to the google drive
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
#switch to the part of your directory
#os.chdir('/content/drive/MyDrive/phiusiil+phishing+url+dataset.zip')
#load the phiusiil+phishing+url+dataset.zip
!unzip -q /content/drive/MyDrive/phiusiil+phishing+url+dataset.zip
#list the unzip file
os.listdir('/content/')

replace PhiUSIIL_Phishing_URL_Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: yes


['.config', 'drive', 'PhiUSIIL_Phishing_URL_Dataset.csv', 'sample_data']

In [27]:
#access the unzipped file
df = pd.read_csv('PhiUSIIL_Phishing_URL_Dataset.csv')  # Update the file name
df.head()


,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,URLSimilarityIndex,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,521848.txt,https://www.southbankmosaics.com,31,www.southbankmosaics.com,24,0,com,100.0,1.000000,0.522907,...,0,0,1,34,20,28,119,0,124,1
1,31372.txt,https://www.uni-mainz.de,23,www.uni-mainz.de,16,0,de,100.0,0.666667,0.032650,...,0,0,1,50,9,8,39,0,217,1
2,597387.txt,https://www.voicefmradio.co.uk,29,www.voicefmradio.co.uk,22,0,uk,100.0,0.866667,0.028555,...,0,0,1,10,2,7,42,2,5,1
3,554095.txt,https://www.sfnmjournal.com,26,www.sfnmjournal.com,19,0,com,100.0,1.000000,0.522907,...,1,1,1,3,27,15,22,1,31,1
4,151578.txt,https://www.rewildingargentina.org,33,www.rewildingargentina.org,26,0,org,100.0,1.000000,0.079963,...,1,0,1,244,15,34,72,1,85,1


In [28]:
#check the value counts
df['label'].value_counts()

,count
label,
1,134850
0,100945


In [29]:
#check for duplicated values
duplicates = df.duplicated().sum()
print(f"Number of duplicated values: {duplicates}")

Number of duplicated values: 0


In [30]:
#check the missing values
missing_values = df.isnull().any()
#condition
if missing_values.any():
    print("There are missing values in the dataset.")
    print("Missing values in each column:")
    print(missing_values)
else:
    print("There are no missing values in the dataset.")

There are no missing values in the dataset.


In [31]:
#columns
print(df.columns)
print(df.info())


Index(['FILENAME', 'URL', 'URLLength', 'Domain', 'DomainLength', 'IsDomainIP',
       'TLD', 'URLSimilarityIndex', 'CharContinuationRate',
       'TLDLegitimateProb', 'URLCharProb', 'TLDLength', 'NoOfSubDomain',
       'HasObfuscation', 'NoOfObfuscatedChar', 'ObfuscationRatio',
       'NoOfLettersInURL', 'LetterRatioInURL', 'NoOfDegitsInURL',
       'DegitRatioInURL', 'NoOfEqualsInURL', 'NoOfQMarkInURL',
       'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL',
       'SpacialCharRatioInURL', 'IsHTTPS', 'LineOfCode', 'LargestLineLength',
       'HasTitle', 'Title', 'DomainTitleMatchScore', 'URLTitleMatchScore',
       'HasFavicon', 'Robots', 'IsResponsive', 'NoOfURLRedirect',
       'NoOfSelfRedirect', 'HasDescription', 'NoOfPopup', 'NoOfiFrame',
       'HasExternalFormSubmit', 'HasSocialNet', 'HasSubmitButton',
       'HasHiddenFields', 'HasPasswordField', 'Bank', 'Pay', 'Crypto',
       'HasCopyrightInfo', 'NoOfImage', 'NoOfCSS', 'NoOfJS', 'NoOfSelfRef',
       'NoOfEmptyRef', 'NoOf

- Leave the Title column as it requires Natural Language Processing. As
much as it is Categorical in Nature.

In [32]:
#categorical column
categoricals = ['TLD']


In [33]:
#convert categorical to numerical
for col in categoricals:
    df[col],_ = pd.factorize(df[col])#convert categorical to numericals just like the Label Encoder

#check if column is converted
df['TLD'].head()

,TLD
0,0
1,1
2,2
3,0
4,3


In [34]:
#drop unnecessary columns
drop_columns = ['Domain','Title','URL','FILENAME']
df = df.drop(drop_columns,axis=1)
df.columns


Index(['URLLength', 'DomainLength', 'IsDomainIP', 'TLD', 'URLSimilarityIndex',
       'CharContinuationRate', 'TLDLegitimateProb', 'URLCharProb', 'TLDLength',
       'NoOfSubDomain', 'HasObfuscation', 'NoOfObfuscatedChar',
       'ObfuscationRatio', 'NoOfLettersInURL', 'LetterRatioInURL',
       'NoOfDegitsInURL', 'DegitRatioInURL', 'NoOfEqualsInURL',
       'NoOfQMarkInURL', 'NoOfAmpersandInURL', 'NoOfOtherSpecialCharsInURL',
       'SpacialCharRatioInURL', 'IsHTTPS', 'LineOfCode', 'LargestLineLength',
       'HasTitle', 'DomainTitleMatchScore', 'URLTitleMatchScore', 'HasFavicon',
       'Robots', 'IsResponsive', 'NoOfURLRedirect', 'NoOfSelfRedirect',
       'HasDescription', 'NoOfPopup', 'NoOfiFrame', 'HasExternalFormSubmit',
       'HasSocialNet', 'HasSubmitButton', 'HasHiddenFields',
       'HasPasswordField', 'Bank', 'Pay', 'Crypto', 'HasCopyrightInfo',
       'NoOfImage', 'NoOfCSS', 'NoOfJS', 'NoOfSelfRef', 'NoOfEmptyRef',
       'NoOfExternalRef', 'label'],
      dtype='object')

## Machine Learning

In [35]:
#define the features and target variable
#feature columns
X = df.drop('label',axis=1)
#target column
y = df['label']

In [36]:
#train test split the data
X_train,X_test,y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42,stratify=y)


In [37]:
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")




X_train shape: (188636, 51)
X_test shape: (47159, 51)
y_train shape: (188636,)
y_test shape: (47159,)


## Build the Models

In [38]:
#initialize the model
rfc = RandomForestClassifier(n_estimators=100, random_state=42)

#fit the model
rfc.fit(X_train,y_train)

RandomForestClassifier(random_state=42)

## Evaluate the Model

In [39]:

#make predictions
y_pred = rfc.predict(X_test)

In [40]:
#compare the actual vs predicted
df_new = pd.DataFrame({'Actual':y_test,'Predicted':y_pred})
df_new.head(10)

,Actual,Predicted
173310,1,1
62486,1,1
14858,1,1
82943,1,1
72539,1,1
191560,0,0
42091,1,1
110432,1,1
90325,1,1
185076,0,0


In [41]:
#accuracy score
accuracy = accuracy_score(y_test,y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 1.0


- Potential risk of overfitting because the accuracy is 100%

In [42]:
#get feature importance. The most important columns
feature_importance = rfc.feature_importances_
#make it a dataframe
feature_importance_df = pd.DataFrame({'Feature':X.columns,'Importance':feature_importance})
#sort the values
feature_importance_df = feature_importance_df.sort_values(by='Importance',ascending=False)

In [43]:
#preview the most important features
feature_importance_df.head(10)

,Feature,Importance
4,URLSimilarityIndex,0.225775
50,NoOfExternalRef,0.181012
23,LineOfCode,0.141393
45,NoOfImage,0.079095
48,NoOfSelfRef,0.064895
47,NoOfJS,0.058850
46,NoOfCSS,0.052615
37,HasSocialNet,0.033848
22,IsHTTPS,0.023611
44,HasCopyrightInfo,0.023184


In [44]:
 #build the model with the most important features
 final = feature_importance_df.head(15)
 final_features = list(final['Feature'])
 final_features


['URLSimilarityIndex',
 'NoOfExternalRef',
 'LineOfCode',
 'NoOfImage',
 'NoOfSelfRef',
 'NoOfJS',
 'NoOfCSS',
 'HasSocialNet',
 'IsHTTPS',
 'HasCopyrightInfo',
 'NoOfOtherSpecialCharsInURL',
 'NoOfDegitsInURL',
 'LargestLineLength',
 'URLLength',
 'DegitRatioInURL']

In [48]:
#assign the columns to X and y
X_new = df[final_features]
y_new = df['label']

#train test split
X_train_new,X_test_new,y_train_new, y_test_new = train_test_split(X_new,y_new,test_size=0.20, random_state=42,stratify=y_new)

In [49]:
#build the model afresh
rfc_new = RandomForestClassifier(n_estimators=100, random_state=42)
rfc_new.fit(X_train_new,y_train_new)


RandomForestClassifier(random_state=42)

In [50]:
#evaluate the model
y_pred_new = rfc_new.predict(X_test_new)

accuracy_new = accuracy_score(y_test_new,y_pred_new)
print(f"Accuracy: {accuracy_new}")


Accuracy: 1.0


- The accuracy is still 100%.